In [17]:
import re
import requests

In [31]:
response = requests.get('https://stsci.app.box.com/s/tj1jnivn9ekiyhecl5up7mkg8xrd1htl/folder/154382715453')
text = response.text

In [32]:
print(text)

 <!DOCTYPE html><html lang="en-US" data-resin-client="web"><head><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="viewport" content="width=device-width, initial-scale=1.0"><meta name="robots" content="noindex, nofollow"><title>Box</title> <link rel="stylesheet" href="https://cdn01.boxcdn.net/fonts/1.0.2/lato/Lato-woff.css"> <link rel="stylesheet" href="https://cdn01.boxcdn.net/enduser/app.5fd82851a5.css">  <link rel="apple-touch-icon" sizes="57x57" href="https://cdn01.boxcdn.net/_assets/img/favicons/apple-touch-icon-57x57-fLlEpj.png">
<link rel="apple-touch-icon" sizes="60x60" href="https://cdn01.boxcdn.net/_assets/img/favicons/apple-touch-icon-60x60-Uv0qzu.png">
<link rel="apple-touch-icon" sizes="72x72" href="https://cdn01.boxcdn.net/_assets/img/favicons/apple-touch-icon-72x72-7aVqne.png">
<link rel="apple-touch-icon" sizes="76x76" href="https://cdn01.boxcdn.net/_assets/img/favicons/apple-touch-icon-76x76-ZVGnRV.png">
<link rel="apple-touch-icon" sizes="114x114" href=

In [45]:
def find_folders(text, parent_folder="d_154382715453"):
    typedIDs = []
    names = []
    locs = [m.start() for m in re.finditer('typedID', text)]
    for l in locs:
        typedID = text[l+10 : l+24]
        if typedID != parent_folder:
            #print(typedID)
            name_loc = text[l:].find('"name":') + l + 8
            name_end_loc = text[name_loc:].find(",") + name_loc - 1
            typedIDs.append(typedID)
            names.append(text[name_loc:name_end_loc])
    return typedIDs, names

folder_typedIDs, folder_names = find_folders(text)
print(folder_typedIDs)
print(folder_names)

['d_154382561036', 'd_154382588636', 'd_154382958627', 'd_154382679630']
['MIRI', 'NIRISS', 'NIRCam', 'NIRSpec']


In [5]:
def find_file_IDs(text):
    #print(text)
    locs = [m.start() for m in re.finditer('typedID', text)]
    all_typedIDs = []
    for l in locs:
        typedID = text[l+10 : l+24]
        if typedID[0] == "f": 
            all_typedIDs.append(typedID)
    #print(l)
    #print(text[locs:locs+25])
    return all_typedIDs

In [27]:
folder_link = 'https://stsci.app.box.com/s/tj1jnivn9ekiyhecl5up7mkg8xrd1htl/folder/155356707447'
page = 1

all_file_IDs = []
while True:    
    response = requests.get('https://stsci.app.box.com/s/tj1jnivn9ekiyhecl5up7mkg8xrd1htl/folder/155356707447?page={}'.format(page))
    file_IDs = find_file_IDs(response.text)
    all_file_IDs += file_IDs
    if len(file_IDs) == 0:
        break   
    print("Read page {} and found file IDs {}".format(page, file_IDs))
    page += 1

Read page 1 and found file IDs ['f_913781875778', 'f_913784798063', 'f_913782068322', 'f_913779537845', 'f_913782020137', 'f_913784399565', 'f_913778758094', 'f_913781756850', 'f_913782058276', 'f_913780911542', 'f_913783187511', 'f_913780565034', 'f_913779733441', 'f_913778132097', 'f_913777802769', 'f_913777272160', 'f_913780703186', 'f_913781482077', 'f_913775070477', 'f_913776712427']
Read page 2 and found file IDs ['f_913780489314', 'f_913776665234', 'f_913773930287', 'f_913777624202', 'f_913779805943', 'f_913776464715', 'f_913775411992', 'f_913775971661', 'f_913774651012', 'f_913773150497', 'f_913773516068', 'f_913775247627', 'f_913772825400', 'f_913776230650', 'f_913775485809', 'f_913775879969', 'f_913776167422', 'f_913773806578', 'f_913773264832', 'f_913771875592']
Read page 3 and found file IDs ['f_913770723975', 'f_913774491166', 'f_913772208331']


In [28]:
shared_name = "tj1jnivn9ekiyhecl5up7mkg8xrd1htl" #Should be the same for the entire repository

for file_ID in all_file_IDs:
    direct_link = "https://app.box.com/index.php?rm=box_download_shared_file&shared_name={}&file_id={}".format(shared_name, file_ID)
    r = requests.get(direct_link)
    filename = re.findall("filename=(.+)", r.headers["Content-Disposition"])[0].split(";")[0].strip('"')
    print("Downloaded file {} with ID {}".format(filename, file_ID))
    with open(filename, "wb") as f:
        f.write(r.content)

KeyboardInterrupt: 